In [7]:
import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import time
from tkinter import TclError

%matplotlib tk

CHUNK = 1024 * 2             # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample)
CHANNELS = 1                 # single channel for microphone
RATE = 44100                 # samples per second

In [ ]:
'''
Convert MP3 files into WAV files

def mp3_to_wav(location, file):
    """Convert MP3 files into WAV files"""
    mp3_file = AudioSegment.from_mp3(os.path.join(location, file) + '.mp3')
    wav_file = os.path.join(location, file) + '.wav'
    mp3_file.export(wav_file, format='wav')
    
    return wav_file

wav_file = mp3_to_wav(location, file)
'''

In [10]:
fig, ax = plt.subplots(1, figsize=(15, 7))

# instantiate PyAudio
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

x_fft = np.linspace(0, RATE, CHUNK)
line_fft, = ax.semilogx(x_fft, np.random.rand(CHUNK), '-', lw=2)

ax.set_title('FREQUENCY SPECTRUM')
ax.set_xlabel('frequency')
ax.set_xlim(20, RATE / 2)

plt.show(block=False)

frame_count = 0
start_time = time.time()

while True:
    
    # binary data
    data = stream.read(CHUNK)  
    
    # convert data to integers, make np array, then offset it by 127
    data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    
    # create np array and offset by 128
    data_np = np.array(data_int, dtype='b')[::2] + 128
    
    line.set_ydata(data_np)
    
    y_fft = fft(data_int)
    line_fft.set_ydata(np.abs(y_fft[0:CHUNK]) * 2 / (256 * CHUNK))
    
    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
    
    # output when the matplotlib window is closed
    except TclError:
        avg_frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(avg_frame_rate))
        break

stream stopped
average frame rate = 20 FPS
